In [2]:
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

# Habitacion 1

queen_bed = {"name": "queen bed",
            "type": "furniture"}
door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}
bomb={
    "name": "bomb",
    "type": "key",
}

#habitacion 2
bedroom_2={
    "name": "bedroom 2",
    "type": "room",

}
double_bed = {
    "name":"double bed",
    "type":"furniture"
}
dresser = {
    "name":"dresser",
    "type":"furniture"
}
door_b = {
    "name": "door b",
    "type": "door"
}

key_c={
    "name":"key for door c",
    "type":"key",
    "target":door_c
}
door_d = {
    "name": "door d",
    "type": "door"
}
key_d={
    "name":"key for door d",
    "type":"key",
    "target":door_d
}
#living room
living_room={
    "name": "living room",
    "type": "room",
}
door_c = {
    "name": "door c",
    "type": "door"
}
dining_table={
    "name":"dining table",
    "type": "furniture"
}
mancha_sangre={
    "name":"mancha sangre",
    "type": "furniture"
}


outside = {
  "name": "outside"
}
all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "outside": [outside],
    "door a": [game_room, bedroom_1],
    "bedroom 1": [queen_bed, door_b,door_c,door_a, mancha_sangre],
    "queen bed": [key_b],
    "door b" : [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "bedroom 2":[double_bed, dresser, door_b],
    "double bed":[key_c],
    "dresser":[key_d],
    "living room":[dining_table, door_c, door_d],
    "door d":[outside],
    "dining table":[bomb]
}
actualiza_inventario = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "outside": [outside],
    "door a": [game_room, bedroom_1],
    "bedroom 1": [queen_bed, door_b,door_c,door_a, mancha_sangre],
    "queen bed": [key_b],
    "door b" : [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "bedroom 2":[double_bed, dresser, door_b],
    "double bed":[key_c],
    "dresser":[key_d],
    "living room":[dining_table, door_c, door_d],
    "door d":[outside],
    "dining table":[bomb]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [3]:
def reset_game():
    object_relations.update(actualiza_inventario)
    game_state["keys_collected"]=[]
    game_state["current_room"]=game_room
    start_game()

def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():


    """
    Start the game

    """
    print("Te despiertas en el sofa de una extraña habitación, piensas que tal vez no deberias de tomarte todas las pastillas que te ofrecen de fiesta, sin ventanas ni ninguna otra salida salvo una puerta, te decides a salir.")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Felicidades!. Has escapado!")
    else:
        print("Tu te encuentras en :  " + room["name"])
        intended_action = input("Que te gustaria hacer? explora o examina: ").strip()
        if intended_action == "explora":
            explore_room(room)
            play_room(room)
        elif intended_action == "examina":
            examine_item(input("¿Que te gustaría examinar?").strip())
        else:
            print("No hay nada de valor, ¿Que te gustaria hacer 'examina' o 'explora'?.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("Tu exploras esta habitacion. Estas en " + room["name"] + ". Encuentras " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "Examinas " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "Has desbloqueado la puerta con la llave."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "Esta puerta esta bloqueada y no tienes la llave..."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    if item_found["name"] =="bomb":
                        print ("has encontrado una puta bomba... le das la vuelta... solo pone quien la deja un tal '1r0n4ack'\n")
                        codigo = input("Introduce el codigo correcto...piensa solo acepta letras... Tic tac..: ")
                        if codigo != "ironhack":
                            print ("has encontrado una puta bomba... con lo cual tu juego aqui ha terminado, pierdes todo lo encontrado... Buena suerte.\n")
                            reset_game()
                    else:
                        game_state["keys_collected"].append(item_found)
                        output += "Encuentras " + item_found["name"] + "."
                else:
                    output += "No hay nada interesante aqui..."
            print(output)
            break

    if(output is None):
        print("el item que seleccionas no está en esta habitación.")

    if(next_room and input("¿Quieres acceder a la siguiente habitación?'si' o 'no' ").strip() == 'si'):
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
eeegame_state = INIT_GAME_STATE.copy()

start_game()

Te despiertas en el sofa de una extraña habitación, piensas que tal vez no deberias de tomarte todas las pastillas que te ofrecen de fiesta, sin ventanas ni ninguna otra salida salvo una puerta, te decides a salir.
Tu te encuentras en :  game room
Que te gustaria hacer? explora o examina: examina
¿Que te gustaría examinar?piano
Examinas piano. Encuentras key for door a.
Tu te encuentras en :  game room
Que te gustaria hacer? explora o examina: examina
¿Que te gustaría examinar?door a
Examinas door a. Has desbloqueado la puerta con la llave.
¿Quieres acceder a la siguiente habitación?'si' o 'no' si
Tu te encuentras en :  bedroom 1
Que te gustaria hacer? explora o examina: xamina
No hay nada de valor, ¿Que te gustaria hacer 'examina' o 'explora'?.
Tu te encuentras en :  bedroom 1
Que te gustaria hacer? explora o examina: examina
¿Que te gustaría examinar?queen bed
Examinas queen bed. Encuentras key for door b.
Tu te encuentras en :  bedroom 1
Que te gustaria hacer? explora o examina: exa